In [1]:
##############################
#03_DataFrame example code   #
#Maintainer: Christopher Chan#
#Version: 0.1.2              #
#Date: 2023-05-10            #
##############################

# Just like we are used to, let's import all our necessary modules here
import os, sys, re
import random
import pathlib

import datetime as dt
import numpy as np
import pyarrow as pa
import pandas as pd


# Let's set the path to where our data are located
data_raw = pathlib.Path("../data/raw")
data_interim = pathlib.Path("../data/interim")
data_external = pathlib.Path("../data/external")
data_processed = pathlib.Path("../data/processed")

## Welcome to 03_DataFrames
This is where your previously learnt class will start to hopefully make sense in a table data environment.
Although manipulating tables and dataframes in python/R and excel has not much differences in outcome,
the massive gain from control, speed, read and write ability justifies moving workflow to any programming language

### 03_DataFrames Goals:
- Read and write DataFrames
- Exploring types in DataFrames
- Indexing and subsetting DataFrames
- Joins, Merge, Pivot

> This is the first part of the Data Science cycle, where we will learn to import and tidy the data

![data-science-explore](../readme_figs/data-science-explore.png)

#### Introducing CSV
If you have always used excels, you realised that excel has a row limit of 1,048,576 until it requires another sheet
If you use any other operating system, you need an excel license to read xlsx.
CSV (aka. Comma Separated Value) is a:
- Platform independent
- Basic
- Row-based dataframe format

As the name suggest values are comma separated, you can open up `../data/external/module_info.csv` to take a look!
The advantages of using CSV:
- Can be edited with any basic text editing software (notepad, excel, word...)
- Much faster read and write in any programming language
- Does not take up all your memory (i.e. No more excel crashes)
- Forces standard format in tables (1 row index, 1 column index)

The disadvantages of using CSV:
- No cool excel like collaborative 
- No excel functions (<-- We can just do all of this in python)
- No colours
- No crazy multi-column, multi-row index support (This forces good practice)

In E&S data, we have a lot of "," in our Answers field, therefore we will use semi-colon ";" as a separator instead!

In [2]:
# Pandas 2.0
# Read our E&S data with 50k samples
ESData_sample = pd.read_csv(f"{data_raw}/ESData_sample.csv", sep = ";")
#ESData_sample = pd.read_csv(f"{data_raw}/ESData_sample.csv", sep = ";", dtype_backend = "pyarrow")

# Full E&S data
ESData = pd.read_parquet(f"{data_raw}/ESData_full.parquet", dtype_backend = "pyarrow", engine = "pyarrow")
date_cols = ESData.select_dtypes(include = "timestamp[us][pyarrow]").columns.tolist()

ESData = ESData.astype({date_cols[0]: "date64[pyarrow]",
                        date_cols[1]: "date64[pyarrow]"})

ESData = ESData.astype({date_cols[0]: "string[pyarrow]",
                        date_cols[1]: "string[pyarrow]"})

print(ESData.size)

# Calling our assigned E&S data automatically gives us the head of the DataFrame and the tail of the DataFrame
ESData_sample

16999983


,FactorId,Name,Question,Date,Answer,PublicationDate,Restated,AgentId,AgentName,AgentGics,CountryOfOperation,SourceName,SourceType,Page,Comment,Excerpt,URL
0,4904,EOPol1.2,Does the company disclose an anti-discriminati...,2021-07-27 00:00:00,Yes,2022-07-28 00:00:00,False,1049,"Expedia Group, Inc.",25301020,"USA,CHN,FRA,AUS,NOR,SWE,DNK,FIN,BEL,GBR,ITA,CH...",EMPLOYEE CODE OF CONDUCT,Policy,6,Wrong Source,We will not tolerate discrimination of any kin...,https://s27.q4cdn.com/708721433/files/doc_down...
1,4946,EMS6,Does the company disclose the number of incide...,2020-12-31 00:00:00,No,2023-02-15 00:00:00,False,844,"S&P Global, Inc.",40203040,"ARE,ARG,AUS,AUT,BEL,BLR,BMU,BRA,BRB,CAN,CHE,CH...",NaN,NaN,NaN,NaN,NaN,NaN
2,4527,WatMon1.2,Does the company disclose details on freshwate...,2021-12-31 00:00:00,Not Meaningful,2023-03-20 00:00:00,False,238,Alpha Services & Holdings SA,40101010,"ALB,BGR,CYP,DEU,GBR,GRC,IRL,JEY,LUX,MKD,ROU,SR...",NaN,NaN,NaN,NaN,NaN,NaN
3,4821,PosNRE2,Does the company disclose measures to mitigate...,2020-12-31 00:00:00,No,2023-03-30 00:00:00,False,736,Abbott Laboratories,35101010,"ARE,ARG,AUS,AUT,BEL,BGD,BGR,BHS,BIH,BMU,BOL,BR...",NaN,NaN,NaN,NaN,NaN,NaN
4,4657,SupSt1.0,Does the company disclose a supplier labour po...,2022-11-01 00:00:00,Yes,2023-03-31 00:00:00,False,1572,Hapag-Lloyd AG,20303010,"DEU,FRA,NLD,BEL,USA,VNM,JPN,MLT,MEX,NZL,GBR,IT...",Hapag-Lloyd Supplier Code of Conduct 2022,Policy (Supplier),1,NaN,• Prohibition of any discrimination based on e...,https://www.hapag-lloyd.com/content/dam/websit...
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
49995,4307,HRDue1.13,Does the company disclose that its suppliers a...,2020-12-31 00:00:00,Yes,2023-03-27 00:00:00,False,887,Texas Instruments Incorporated,45301020,"USA,CAN,ISR,DNK,NLD,DEU,FRA,GBR,PHL,SGP,MYS,KO...",2020 CORPORATE CITIZENSHIP REPORT,Sustainability Report,32,NaN,To manage human rights and eliminate violation...,https://www.ti.com/lit/ml/szzo015/szzo015.pdf?...
49996,4541,WatInv2.5,Does the company disclose metrics for withdraw...,2021-12-31 00:00:00,Yes,2023-03-04 00:00:00,False,556,Electricite de France SA,55101010,"AUS,BEL,BGR,BRA,CAN,CHE,CHL,CHN,CUW,CZE,DEU,DJ...","EDF group's Environmental, Social and Governan...",Company website,NaN,NaN,E-Water resources management (sheet)\nDrinking...,https://www.edf.fr/sites/groupe/files/2022-04/...
49997,5026,WatInv4.0-x,Total water withdrawn from areas with high wat...,2021-12-31 00:00:00,7.339262 m Cubic metre (m3),2023-02-23 00:00:00,False,854,"NextEra Energy, Inc.",55101010,"USA,LKA,NLD,CAN,CYM,ESP",2022 ESG Report,Sustainability Report,62,NaN,percentage of\neach in regions of high or extr...,https://www.investor.nexteraenergy.com/~/media...
49998,4583,HRSup5.0,Does the company express an expectation on sup...,2021-12-31 00:00:00,Not Meaningful,2023-04-24 00:00:00,False,1125,Las Vegas Sands Corp.,25301010,"BMU,CHN,CYM,HKG,IND,JPN,KOR,MAC,MUS,MYS,NLD,PR...",NaN,NaN,NaN,NaN,NaN,NaN


In [3]:
# Let's make sure our sample have 50,000 rows
print(ESData_sample.shape)
print(f"Our sample E&S DataFrame have {ESData_sample.shape[0]} rows, and {ESData_sample.shape[1]} columns!")

(50000, 17)
Our sample E&S DataFrame have 50000 rows, and 17 columns!


### We should probably explore the data a little bit more...
- Look at it
- Find some things about it
- Look at the data type

In [4]:
# Look at the heads and tails, and random samples
ESData_sample.head()

,FactorId,Name,Question,Date,Answer,PublicationDate,Restated,AgentId,AgentName,AgentGics,CountryOfOperation,SourceName,SourceType,Page,Comment,Excerpt,URL
0,4904,EOPol1.2,Does the company disclose an anti-discriminati...,2021-07-27 00:00:00,Yes,2022-07-28 00:00:00,False,1049,"Expedia Group, Inc.",25301020,"USA,CHN,FRA,AUS,NOR,SWE,DNK,FIN,BEL,GBR,ITA,CH...",EMPLOYEE CODE OF CONDUCT,Policy,6,Wrong Source,We will not tolerate discrimination of any kin...,https://s27.q4cdn.com/708721433/files/doc_down...
1,4946,EMS6,Does the company disclose the number of incide...,2020-12-31 00:00:00,No,2023-02-15 00:00:00,False,844,"S&P Global, Inc.",40203040,"ARE,ARG,AUS,AUT,BEL,BLR,BMU,BRA,BRB,CAN,CHE,CH...",NaN,NaN,NaN,NaN,NaN,NaN
2,4527,WatMon1.2,Does the company disclose details on freshwate...,2021-12-31 00:00:00,Not Meaningful,2023-03-20 00:00:00,False,238,Alpha Services & Holdings SA,40101010,"ALB,BGR,CYP,DEU,GBR,GRC,IRL,JEY,LUX,MKD,ROU,SR...",NaN,NaN,NaN,NaN,NaN,NaN
3,4821,PosNRE2,Does the company disclose measures to mitigate...,2020-12-31 00:00:00,No,2023-03-30 00:00:00,False,736,Abbott Laboratories,35101010,"ARE,ARG,AUS,AUT,BEL,BGD,BGR,BHS,BIH,BMU,BOL,BR...",NaN,NaN,NaN,NaN,NaN,NaN
4,4657,SupSt1.0,Does the company disclose a supplier labour po...,2022-11-01 00:00:00,Yes,2023-03-31 00:00:00,False,1572,Hapag-Lloyd AG,20303010,"DEU,FRA,NLD,BEL,USA,VNM,JPN,MLT,MEX,NZL,GBR,IT...",Hapag-Lloyd Supplier Code of Conduct 2022,Policy (Supplier),1,NaN,• Prohibition of any discrimination based on e...,https://www.hapag-lloyd.com/content/dam/websit...


In [5]:
ESData_sample.tail()

,FactorId,Name,Question,Date,Answer,PublicationDate,Restated,AgentId,AgentName,AgentGics,CountryOfOperation,SourceName,SourceType,Page,Comment,Excerpt,URL
49995,4307,HRDue1.13,Does the company disclose that its suppliers a...,2020-12-31 00:00:00,Yes,2023-03-27 00:00:00,False,887,Texas Instruments Incorporated,45301020,"USA,CAN,ISR,DNK,NLD,DEU,FRA,GBR,PHL,SGP,MYS,KO...",2020 CORPORATE CITIZENSHIP REPORT,Sustainability Report,32,NaN,To manage human rights and eliminate violation...,https://www.ti.com/lit/ml/szzo015/szzo015.pdf?...
49996,4541,WatInv2.5,Does the company disclose metrics for withdraw...,2021-12-31 00:00:00,Yes,2023-03-04 00:00:00,False,556,Electricite de France SA,55101010,"AUS,BEL,BGR,BRA,CAN,CHE,CHL,CHN,CUW,CZE,DEU,DJ...","EDF group's Environmental, Social and Governan...",Company website,NaN,NaN,E-Water resources management (sheet)\nDrinking...,https://www.edf.fr/sites/groupe/files/2022-04/...
49997,5026,WatInv4.0-x,Total water withdrawn from areas with high wat...,2021-12-31 00:00:00,7.339262 m Cubic metre (m3),2023-02-23 00:00:00,False,854,"NextEra Energy, Inc.",55101010,"USA,LKA,NLD,CAN,CYM,ESP",2022 ESG Report,Sustainability Report,62,NaN,percentage of\neach in regions of high or extr...,https://www.investor.nexteraenergy.com/~/media...
49998,4583,HRSup5.0,Does the company express an expectation on sup...,2021-12-31 00:00:00,Not Meaningful,2023-04-24 00:00:00,False,1125,Las Vegas Sands Corp.,25301010,"BMU,CHN,CYM,HKG,IND,JPN,KOR,MAC,MUS,MYS,NLD,PR...",NaN,NaN,NaN,NaN,NaN,NaN
49999,4680,SupSt1.104,Does the company disclose a supplier policy pr...,2022-08-31 00:00:00,No,2023-02-03 00:00:00,False,738,Accenture Plc,45102010,"AGO,AND,ARE,ARG,AUS,AUT,BEL,BGD,BGR,BMU,BOL,BR...",NaN,NaN,NaN,NaN,NaN,NaN


In [6]:
# Random sample of 4
ESData_sample.sample(n = 4)

,FactorId,Name,Question,Date,Answer,PublicationDate,Restated,AgentId,AgentName,AgentGics,CountryOfOperation,SourceName,SourceType,Page,Comment,Excerpt,URL
43298,5254,HAcc2.6,Does the company disclose the aggregate lost t...,2021-12-31 00:00:00,No,2023-04-20 00:00:00,False,735,3M Company,20105010,"USA,DEU,CAN,JAM,TTO,GBR,DNK,AUS,ITA,FRA,JPN,SV...",NaN,NaN,NaN,NaN,NaN,NaN
11257,4881,FoAPol4,Does the company disclose a commitment to proh...,2022-07-14 00:00:00,"Yes, in a policy",2023-03-28 00:00:00,False,648,Publicis Groupe SA,50201010,"FRA,USA,SWE,GBR,ITA,PHL,DEU,NZL,CAN,NOR,LVA,IS...","CORPORATE SOCIAL RESPONSIBILITY, SUSTAINABILIT...",Policy,"1,2",NaN,The Groupe commits to the United\nNations Glob...,https://publicisgroupe-csr-smart-data.com/asse...
37076,5070,EMS5-x,Number of incidents of non-compliance with air...,2021-06-30 00:00:00,Not Disclosed,2023-03-03 00:00:00,False,991,The Clorox Company,30301010,"USA,CHL,CAN,ARG,COL,PRI,GBR,LUX,NLD,EGY,MEX,AU...",NaN,NaN,NaN,NaN,NaN,NaN
32028,4518,WatStr1.20,Does the company disclose risk evaluation that...,2021-12-31 00:00:00,Not Meaningful,2023-04-04 00:00:00,False,308,Fresenius Medical Care AG & Co. KGaA,35102015,"ARE,ARG,AUS,BEL,BGR,BIH,BMU,BRA,CAN,CHE,CHN,CO...",NaN,NaN,NaN,NaN,NaN,NaN


In [7]:
# We can access individual column and multiple columns using
ESData_sample["Question"]

0        Does the company disclose an anti-discriminati...
1        Does the company disclose the number of incide...
2        Does the company disclose details on freshwate...
3        Does the company disclose measures to mitigate...
4        Does the company disclose a supplier labour po...
                               ...                        
49995    Does the company disclose that its suppliers a...
49996    Does the company disclose metrics for withdraw...
49997    Total water withdrawn from areas with high wat...
49998    Does the company express an expectation on sup...
49999    Does the company disclose a supplier policy pr...
Name: Question, Length: 50000, dtype: object

In [8]:
#Multiple Columns
ESData_sample[["FactorId", "Question"]]

,FactorId,Question
0,4904,Does the company disclose an anti-discriminati...
1,4946,Does the company disclose the number of incide...
2,4527,Does the company disclose details on freshwate...
3,4821,Does the company disclose measures to mitigate...
4,4657,Does the company disclose a supplier labour po...
...,...,...
49995,4307,Does the company disclose that its suppliers a...
49996,4541,Does the company disclose metrics for withdraw...
49997,5026,Total water withdrawn from areas with high wat...
49998,4583,Does the company express an expectation on sup...


In [9]:
# Describe and data type info
# This finds the numeric columns automatically and give us 
ESData_sample.describe()

,FactorId,AgentId,AgentGics
count,50000.000000,50000.000000,5.000000e+04
mean,4725.556900,713.900940,3.212140e+07
std,236.589686,394.393798,1.412414e+07
min,4244.000000,1.000000,1.010101e+07
25%,4558.000000,404.000000,2.010602e+07
50%,4743.000000,702.000000,3.020203e+07
75%,4919.000000,1022.000000,4.030105e+07
max,5255.000000,1598.000000,6.020103e+07


In [10]:
# Data Types that we learnt
# We see that Date and PublicationDate is not a date object yet, we can change that!
print(ESData_sample.info())

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 50000 entries, 0 to 49999
Data columns (total 17 columns):
 #   Column              Non-Null Count  Dtype 
---  ------              --------------  ----- 
 0   FactorId            50000 non-null  int64 
 1   Name                50000 non-null  object
 2   Question            50000 non-null  object
 3   Date                50000 non-null  object
 4   Answer              49998 non-null  object
 5   PublicationDate     50000 non-null  object
 6   Restated            50000 non-null  bool  
 7   AgentId             50000 non-null  int64 
 8   AgentName           50000 non-null  object
 9   AgentGics           50000 non-null  int64 
 10  CountryOfOperation  50000 non-null  object
 11  SourceName          18815 non-null  object
 12  SourceType          18815 non-null  object
 13  Page                16514 non-null  object
 14  Comment             2497 non-null   object
 15  Excerpt             17230 non-null  object
 16  URL                 18

In [11]:
# Let's change the columns data type
ESData_sample[["Date", "PublicationDate"]] = ESData_sample[["Date", "PublicationDate"]].apply(pd.to_datetime)

print(ESData_sample[["Date", "PublicationDate"]].dtypes)

ESData_sample[["Date", "PublicationDate"]].sample(n = 8)

Date               datetime64[ns]
PublicationDate    datetime64[ns]
dtype: object


,Date,PublicationDate
46134,2022-12-31,2023-03-28
29830,2021-12-31,2023-03-29
22440,2020-12-31,2022-12-22
10094,2020-12-31,2023-03-24
10366,2021-12-31,2023-03-06
41549,2021-09-30,2023-01-31
240,2020-12-31,2022-12-22
4156,2021-12-31,2022-07-07


### Subsetting Dataframes using Location and Integer-Location (loc, iloc)
- loc: indexing dataframes using names
    - This is useful when we have both row names/index and column names

- iloc: indexing dataframes using integer locations
    - This is useful when we count rows and columns

In [12]:
# loc

# We can select 1 specific row, and 1 specific column by name
print(ESData_sample.loc[999, "Question"])

# We can select a range of rows and a range of columns by name
print(ESData_sample.loc[997:999, "FactorId":"Question"])

# So why does this give us an error?
print(ESData_sample.loc[-1, "FactorId":"Question"])


Does the company disclose the existence of an anonymous hotline for supply chain workers?
     FactorId       Name                                           Question
997      4927  EOMsr1.23  Does the company disclose ethnic/racial divers...
998      4972    TraM2-x  Percentage of employees receiving career devel...
999      4778  SupCP4.11  Does the company disclose the existence of an ...


KeyError: -1

In [ ]:
# iloc

# We can achieve the same result of location with iloc
print(ESData_sample.iloc[999, 2])

# Note that in iloc we use 997:1000 instead of loc[997:999]
# This is because pandas with iloc now uses integer counting method instead of pure indexing
print(ESData_sample.iloc[997:1000, 0:3])

# But now we can also do negatives
print(ESData_sample.iloc[-1, 0:3])

Does the company disclose the existence of an anonymous hotline for supply chain workers?
     FactorId       Name                                           Question
997      4927  EOMsr1.23  Does the company disclose ethnic/racial divers...
998      4972    TraM2-x  Percentage of employees receiving career devel...
999      4778  SupCP4.11  Does the company disclose the existence of an ...
FactorId                                                 4680
Name                                               SupSt1.104
Question    Does the company disclose a supplier policy pr...
Name: 49999, dtype: object


### Let's take a look at a specific company perhaps?
Mercedes-Benz?

In [13]:
Mercedes_df = ESData[ESData["AgentName"] == "Mercedes-Benz Group AG"]
del(ESData)

Mercedes_df.sample(n = 5)

,FactorId,Name,Question,Date,Answer,PublicationDate,Restated,AgentId,AgentName,AgentGics,CountryOfOperation,SourceName,SourceType,Page,Comment,Excerpt,URL
368787,4845,CoC2.11,Does the company embed in a public policy rest...,2023-01-25,Yes,2023-04-26,False,540,Mercedes-Benz Group AG,25102010,"ARE,ARG,AUS,AUT,BEL,BGR,BRA,CAN,CHE,CHN,COL,CZ...",Our Integrity Code [1/25/23],Policy,21,<NA>,Also when giving gifts and invitations to our ...,https://group.mercedes-benz.com/documents/comp...
368595,4878,HInt4,Does the company disclose detailed information...,2022-12-31,No,2023-04-26,False,540,Mercedes-Benz Group AG,25102010,"ARE,ARG,AUS,AUT,BEL,BGR,BRA,CAN,CHE,CHN,COL,CZ...",<NA>,<NA>,<NA>,<NA>,<NA>,<NA>
368414,5081,CliTar1.51-x,Percentage GHG reduction target – Scope 1+2+3,2022-12-31,Not Disclosed,2023-04-26,False,540,Mercedes-Benz Group AG,25102010,"ARE,ARG,AUS,AUT,BEL,BGR,BRA,CAN,CHE,CHN,COL,CZ...",<NA>,<NA>,<NA>,<NA>,<NA>,<NA>
368398,4998,CliInv25-x,Total scope 3 emissions for Category 11 - Use ...,2021-12-31,78.2 m Metric tonnes (t) CO2e,2023-04-26,True,540,Mercedes-Benz Group AG,25102010,"ARE,ARG,AUS,AUT,BEL,BGR,BRA,CAN,CHE,CHN,COL,CZ...",Sustainability Report 2022,Sustainability Report,103,<NA>,Scope 3 Specific Absolute CO2 in million t Use...,https://sustainabilityreport.mercedes-benz.com...
368297,4601,CliInv4.2,Does the company disclose its market-based sco...,2022-12-31,Yes,2023-04-26,False,540,Mercedes-Benz Group AG,25102010,"ARE,ARG,AUS,AUT,BEL,BGR,BRA,CAN,CHE,CHN,COL,CZ...",Sustainability Report 2022,Sustainability Report,116,<NA>,CO2 indirect (Scope 2) — Market-based,https://sustainabilityreport.mercedes-benz.com...


### Writing and saving data

In [14]:
# Let's save our Mercedes_df subset as excel
with open(f"{data_processed}/Mercedes_df.xlsx", "wb") as merc:
    Mercedes_df.to_excel(merc, index = False)

# Save as csv
with open(f"{data_processed}/Mercedes_df.csv", "wb") as merc:
    Mercedes_df.to_csv(merc, sep = ";", index = False)

We can now read our newly created Mercedes CSV! <br>
Woops, this way of reading the file is different! <br>
There seems to be a few ways to open a file!

In [15]:
with open(f"{data_processed}/Mercedes_df.csv", "rb") as merc:
    Mercedes_df = pd.read_csv(merc, dtype_backend = "pyarrow", sep = ";")

Mercedes_df.sample(n = 5)

,FactorId,Name,Question,Date,Answer,PublicationDate,Restated,AgentId,AgentName,AgentGics,CountryOfOperation,SourceName,SourceType,Page,Comment,Excerpt,URL
1245,4842,CoPro6,Does the company disclose an anti-corruption d...,2022-12-31,No,2023-04-26,False,540,Mercedes-Benz Group AG,25102010,"ARE,ARG,AUS,AUT,BEL,BGR,BRA,CAN,CHE,CHN,COL,CZ...",<NA>,<NA>,<NA>,<NA>,<NA>,<NA>
1287,4658,SupSt1.1,Does the company disclose a contractually bind...,2022-07-22,Yes,2023-04-26,False,540,Mercedes-Benz Group AG,25102010,"ARE,ARG,AUS,AUT,BEL,BGR,BRA,CAN,CHE,CHN,COL,CZ...",Responsible Sourcing Standards 2022,Policy (Supplier),6,<NA>,If the Partner does not meet the requirements ...,https://supplier.mercedes-benz.com/servlet/Jiv...
984,4470,WLB10.1,Does the company disclose leave entitlements?,2022-12-31,Yes,2023-04-26,False,540,Mercedes-Benz Group AG,25102010,"ARE,ARG,AUS,AUT,BEL,BGR,BRA,CAN,CHE,CHN,COL,CZ...",Sustainability Report 2022,Sustainability Report,160,<NA>,"Furthermore, the Mercedes-Benz Group also supp...",https://sustainabilityreport.mercedes-benz.com...
424,4664,SupSt1.51,Does the company disclose a supplier policy th...,2022-06-21,Yes,2023-04-26,False,540,Mercedes-Benz Group AG,25102010,"ARE,ARG,AUS,AUT,BEL,BGR,BRA,CAN,CHE,CHN,COL,CZ...",Responsible Sourcing Standards,Policy,<NA>,<NA>,<NA>,https://supplier-portal.daimler.com/servlet/Ji...
794,4637,CliTar3,Does the company describe GHG emissions reduct...,2022-12-31,Yes,2023-04-26,False,540,Mercedes-Benz Group AG,25102010,"ARE,ARG,AUS,AUT,BEL,BGR,BRA,CAN,CHE,CHN,COL,CZ...",Sustainability Report 2022,Sustainability Report,73,<NA>,"CO2 reduction Since 2022, all of Mercedes-Ben...",https://sustainabilityreport.mercedes-benz.com...


#### Duplications, Check for duplications!

In [16]:
ESData_sample.loc[ESData_sample.duplicated()]

,FactorId,Name,Question,Date,Answer,PublicationDate,Restated,AgentId,AgentName,AgentGics,CountryOfOperation,SourceName,SourceType,Page,Comment,Excerpt,URL


### Let's do some more complicated things
Now, that we know the basics of reading a dataframe, let's do some joining! <br>
Joinings or Mergings are combining 2 tables based on similar values!

![joins](../readme_figs/joins.jpg)

So, we have some information about the Published Companies Domicile information we have not included, <br>
But, it is in another xlsx file, it is in the external folder!

In [17]:
Domicile_df = pd.read_excel(f"{data_external}/PublishedCompanies_2023-04-18.xlsx")

Domicile_df.sample(n = 5)

,Agent ID,ISS Company ID,Agent,Domicile
5471,25890,568427,Vanda Pharmaceuticals Inc.,United States
3575,5268,557497,Nexity SA,France
1318,79057,105055,"Cornerstone Building Brands, Inc.",United States
199,811445,526837,Alerus Financial Corporation,United States
3788,952872,3344808,Open Lending Corp.,United States


We have new information that we wanted to include into the E&S raw data! <br>
We have 1 column that is common between both dataframes: Agent Id.

> Perform a left join on AgentId!

![left_join](../readme_figs/left_join.jpg)

In [18]:
# https://pandas.pydata.org/pandas-docs/stable/reference/api/pandas.merge.html
ESSample_wDomLEFT = pd.merge(ESData_sample, Domicile_df, how = "left", left_on = "AgentId", right_on = "Agent ID")
ESSample_wDomRIGHT = pd.merge(ESData_sample, Domicile_df, how = "right", left_on = "AgentId", right_on = "Agent ID")

ESSample_wDomLEFT.sample(n = 5)

,FactorId,Name,Question,Date,Answer,PublicationDate,Restated,AgentId,AgentName,AgentGics,...,SourceName,SourceType,Page,Comment,Excerpt,URL,Agent ID,ISS Company ID,Agent,Domicile
46961,4962,Client1-x,Absenteeism rate,2021-12-31,Not Disclosed,2022-12-07,False,394,Piraeus Financial Holdings SA,40101010,...,NaN,NaN,NaN,NaN,NaN,NaN,394.0,503038.0,Piraeus Financial Holdings SA,Greece
49559,4276,HRPol0.3,Does the company express a commitment to adher...,2020-12-31,No,2023-01-21,False,1048,"Everest Re Group, Ltd.",40301050,...,NaN,NaN,NaN,NaN,NaN,NaN,1048.0,50466.0,"Everest Re Group, Ltd.",Bermuda
43065,5029,WatInv4.141-x,Quantity withdrawn from surface water,2021-06-30,Not Disclosed,2023-03-13,False,969,"Cardinal Health, Inc.",35102010,...,NaN,NaN,NaN,NaN,NaN,NaN,969.0,24785.0,"Cardinal Health, Inc.",United States
8331,4471,WLB10.0,Do the company’s entitlements include special ...,2021-12-31,No,2023-03-10,False,541,Danone SA,30202030,...,NaN,NaN,NaN,NaN,NaN,NaN,541.0,66613.0,Danone SA,France
30794,4753,SupCP2.03,Does the company disclose the existence of sup...,2021-12-31,No,2023-04-19,False,466,VERBUND AG,55101010,...,NaN,NaN,NaN,NaN,NaN,NaN,466.0,510377.0,VERBUND AG,Austria


In [19]:
ESSample_wDomRIGHT.sample(n = 5)

,FactorId,Name,Question,Date,Answer,PublicationDate,Restated,AgentId,AgentName,AgentGics,...,SourceName,SourceType,Page,Comment,Excerpt,URL,Agent ID,ISS Company ID,Agent,Domicile
21380,4266.0,ComOut1.6,Does the company refer to an unspecified commu...,2020-12-31,Not Meaningful,2022-12-22,False,1573.0,IKB Deutsche Industriebank AG,40101010.0,...,NaN,NaN,NaN,NaN,NaN,NaN,1573,509540,IKB Deutsche Industriebank AG,Germany
2386,4304.0,HRDue1.1,Does the company disclose the groups of rights...,2022-03-31,Yes,2022-12-29,False,486.0,Alstom SA,20106010.0,...,UNIVERSAL REGISTRATION DOCUMENT 2021/22,Annual Report,309,NaN,The analysis and prioritisation of human right...,https://www.alstom.com/sites/alstom.com/files/...,486,514827,Alstom SA,France
36116,5068.0,Hw1-x,Total hazardous waste,2022-12-31,16.5 Kilotonne [Metric],2023-04-13,False,406.0,Rolls-Royce Holdings Plc,20101010.0,...,2022 Annual Report,Annual Report,38,NaN,The total amount of solid and liquid waste gen...,https://www.rolls-royce.com/~/media/Files/R/Ro...,406,132328,Rolls-Royce Holdings Plc,United Kingdom
5154,5006.0,ECer1-x,Percentage of company certified to ISO 14001,2022-12-31,Not Disclosed,2023-04-14,False,500.0,Aviva Plc,40301030.0,...,NaN,NaN,NaN,NaN,NaN,NaN,500,509405,Aviva Plc,United Kingdom
40246,4525.0,WatStr1.53,Does the company disclose the use of establish...,2021-10-03,No,2023-03-21,False,1280.0,Starbucks Corporation,25301040.0,...,NaN,NaN,NaN,NaN,NaN,NaN,1280,146082,Starbucks Corporation,United States


### Let's go back to our Mercedes data!
- Can we find out more about it?
- Let's focus on Environmental factors?

In [20]:
# Let's make a list of relevant factors:
Clim_factors = [4980, 4981, 4982, 4983, 4984, 4985, 4986, 4987, 4988, 4989,
                4990, 4991, 4992, 4993, 4994, 4995, 4996, 4997, 4998, 4999,
                5000, 5001, 5002, 5077, 5078, 5079, 5080, 5081, 5082, 5003,
                5055, 5056, 5057, 5058, 5059, 5083, 4963]


Merc_Clim = Mercedes_df[Mercedes_df["FactorId"].isin(Clim_factors)]
print(Merc_Clim["Question"].unique())
Merc_Clim.sample(n = 5)

<ArrowExtensionArray>
[                                       'Percentage of company certified to ISO 50001',
                                                         'Total scope 1 GHG emissions',
                                        'Total scope 2 GHG emissions (location-based)',
                                          'Total scope 2 GHG emissions (market-based)',
                                           'Total scope 2 GHG emissions (unspecified)',
                                      'Total scope 1+2 GHG emissions (location-based)',
                                        'Total scope 1+2 GHG emissions (market-based)',
                                         'Total scope 1+2 GHG emissions (unspecified)',
                                                         'Total scope 3 GHG emissions',
               'Total scope 3 emissions for Category 1 - Purchased goods and services',
                              'Total scope 3 emissions for Category 2 - Capital goods',
        'T

,FactorId,Name,Question,Date,Answer,PublicationDate,Restated,AgentId,AgentName,AgentGics,CountryOfOperation,SourceName,SourceType,Page,Comment,Excerpt,URL
862,5000,CliInv27-x,Total scope 3 emissions for Category 13 - Down...,2022-12-31,Not Disclosed,2023-04-26,False,540,Mercedes-Benz Group AG,25102010,"ARE,ARG,AUS,AUT,BEL,BGR,BRA,CAN,CHE,CHN,COL,CZ...",<NA>,<NA>,<NA>,<NA>,<NA>,<NA>
872,5078,CliTar1.21-x,Percentage GHG reduction target – Scope 2,2022-12-31,Not Disclosed,2023-04-26,False,540,Mercedes-Benz Group AG,25102010,"ARE,ARG,AUS,AUT,BEL,BGR,BRA,CAN,CHE,CHN,COL,CZ...",<NA>,<NA>,<NA>,<NA>,<NA>,<NA>
831,4983,CliInv4.3-x,Total scope 2 GHG emissions (unspecified),2022-12-31,Not Disclosed,2023-04-26,False,540,Mercedes-Benz Group AG,25102010,"ARE,ARG,AUS,AUT,BEL,BGR,BRA,CAN,CHE,CHN,COL,CZ...",<NA>,<NA>,<NA>,<NA>,<NA>,<NA>
155,4981,CliInv4.1-x,Total scope 2 GHG emissions (location-based),2020-12-31,1.492 m Metric tonnes (t) CO2,2023-04-26,False,540,Mercedes-Benz Group AG,25102010,"ARE,ARG,AUS,AUT,BEL,BGR,BRA,CAN,CHE,CHN,COL,CZ...",Sustainability Report,Sustainability Report,153,<NA>,CO2 indirect (Scope 2) — location-based,https://group.mercedes-benz.com/documents/sust...
857,4997,CliInv24-x,Total scope 3 emissions for Category 10 - Proc...,2022-12-31,Not Disclosed,2023-04-26,False,540,Mercedes-Benz Group AG,25102010,"ARE,ARG,AUS,AUT,BEL,BGR,BRA,CAN,CHE,CHN,COL,CZ...",<NA>,<NA>,<NA>,<NA>,<NA>,<NA>


### Can we refine for TCFD disclosure? (Scopes measurements in questions?)
#### Introducing Regular Expression (aka. regex / grep / alienspeak)!
![regex](../readme_figs/regex.jpg)

Regular Expression, first invented in 1951, was a way of classifying and pattern matching languages.
This was the defacto way search egines were built before the existence of NLP based methods

For the example, let's say we want to query for "scope" follow by any numer [1, 2, 3] in the Mercedes dataframe question.
**Scope specified disclosure**
This means we have to match:
- scope 3
- scope 1+2

In [21]:
Merc_TCFD = Merc_Clim[Merc_Clim["Question"].str.contains(r"scope\s\d.*", regex = True)]

# Let's check our query results
print(Merc_TCFD["Question"].unique())

<ArrowExtensionArray>
[                                                        'Total scope 1 GHG emissions',
                                        'Total scope 2 GHG emissions (location-based)',
                                          'Total scope 2 GHG emissions (market-based)',
                                           'Total scope 2 GHG emissions (unspecified)',
                                      'Total scope 1+2 GHG emissions (location-based)',
                                        'Total scope 1+2 GHG emissions (market-based)',
                                         'Total scope 1+2 GHG emissions (unspecified)',
                                                         'Total scope 3 GHG emissions',
               'Total scope 3 emissions for Category 1 - Purchased goods and services',
                              'Total scope 3 emissions for Category 2 - Capital goods',
        'Total scope 3 emissions for Category 3 - Fuel- and energy-related activities',
   'Total 

In [22]:
Merc_TCFD.sample(n = 5)

,FactorId,Name,Question,Date,Answer,PublicationDate,Restated,AgentId,AgentName,AgentGics,CountryOfOperation,SourceName,SourceType,Page,Comment,Excerpt,URL
839,4987,CliInv30-x,Total scope 3 GHG emissions,2022-12-31,97.8 m Metric tonnes (t) CO2e,2023-04-26,False,540,Mercedes-Benz Group AG,25102010,"ARE,ARG,AUS,AUT,BEL,BGR,BRA,CAN,CHE,CHN,COL,CZ...",Sustainability Report 2022,Sustainability Report,103,<NA>,Scope 3 Specific Absolute CO2 in million t Pro...,https://sustainabilityreport.mercedes-benz.com...
178,4998,CliInv25-x,Total scope 3 emissions for Category 11 - Use ...,2021-12-31,99 m Metric tonnes (t) CO2,2023-04-26,False,540,Mercedes-Benz Group AG,25102010,"ARE,ARG,AUS,AUT,BEL,BGR,BRA,CAN,CHE,CHN,COL,CZ...",Sustainability Report,Sustainability Report,140,<NA>,Scope 3 (r Mercedes-Benz Cars1) Absolute sco...,https://group.mercedes-benz.com/documents/sust...
848,4992,CliInv19-x,Total scope 3 emissions for Category 5 - Waste...,2022-12-31,800000 Metric tonnes (t) CO2e,2023-04-26,False,540,Mercedes-Benz Group AG,25102010,"ARE,ARG,AUS,AUT,BEL,BGR,BRA,CAN,CHE,CHN,COL,CZ...",Sustainability Report 2022,Sustainability Report,103,<NA>,Scope 3 Specific Absolute CO2 in million t Dis...,https://sustainabilityreport.mercedes-benz.com...
177,4997,CliInv24-x,Total scope 3 emissions for Category 10 - Proc...,2021-12-31,Not Disclosed,2023-04-26,False,540,Mercedes-Benz Group AG,25102010,"ARE,ARG,AUS,AUT,BEL,BGR,BRA,CAN,CHE,CHN,COL,CZ...",<NA>,<NA>,<NA>,<NA>,<NA>,<NA>
857,4997,CliInv24-x,Total scope 3 emissions for Category 10 - Proc...,2022-12-31,Not Disclosed,2023-04-26,False,540,Mercedes-Benz Group AG,25102010,"ARE,ARG,AUS,AUT,BEL,BGR,BRA,CAN,CHE,CHN,COL,CZ...",<NA>,<NA>,<NA>,<NA>,<NA>,<NA>


### Pivot tables
Sometimes, we want to make the row values into columns and for detail comparison.
Now that we have the TCFD scopes of Mercedes-Benz, let's make the Question into columns and their Answers as values

In [87]:
# Pivot tables
# Wow this got a bit complicated, don't worry this is exceptional
# But let's just visualise the result, we can learn about lambda later.
Merc_TCFDPivot = pd.pivot_table(Merc_TCFD, values = "Answer", columns = "Question", index = "Date", aggfunc = lambda x: " ".join(x))

# Nice!
# Let's save it
with open(f"{data_processed}/Merc_TCFD.csv", "wb") as data:
    Merc_TCFDPivot.to_csv(data, sep = ";")

Merc_TCFDPivot

Question,Total scope 1 GHG emissions,Total scope 1+2 GHG emissions (location-based),Total scope 1+2 GHG emissions (market-based),Total scope 1+2 GHG emissions (unspecified),Total scope 2 GHG emissions (location-based),Total scope 2 GHG emissions (market-based),Total scope 2 GHG emissions (unspecified),Total scope 3 GHG emissions,Total scope 3 emissions for Category 1 - Purch...,Total scope 3 emissions for Category 10 - Proc...,...,Total scope 3 emissions for Category 14 - Fran...,Total scope 3 emissions for Category 15 - Inve...,Total scope 3 emissions for Category 2 - Capit...,Total scope 3 emissions for Category 3 - Fuel-...,Total scope 3 emissions for Category 4 - Upstr...,Total scope 3 emissions for Category 5 - Waste...,Total scope 3 emissions for Category 6 - Busin...,Total scope 3 emissions for Category 7 - Emplo...,Total scope 3 emissions for Category 8 - Upstr...,Total scope 3 emissions for Category 9 - Downs...
Date,,,,,,,,,,,,,,,,,,,,,
2019-12-31,1.239 m Metric tonnes (t) CO2,2.946 m Metric tonnes (t) CO2,2.516 m Metric tonnes (t) CO2,<NA>,1.706 m Metric tonnes (t) CO2,1.276 m Metric tonnes (t) CO2,<NA>,<NA>,<NA>,<NA>,...,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>
2020-12-31,1.027 m Metric tonnes (t) CO2,2.519 m Metric tonnes (t) CO2,2.062 m Metric tonnes (t) CO2,<NA>,1.492 m Metric tonnes (t) CO2,1.035 m Metric tonnes (t) CO2,<NA>,103.2 m Metric tonnes (t) CO2e,17 m Metric tonnes (t) CO2e,<NA>,...,<NA>,<NA>,<NA>,<NA>,<NA>,800000 Metric tonnes (t) CO2e,12000 Metric tonnes (t) CO2e,125000 Metric tonnes (t) CO2e,<NA>,<NA>
2021-12-31,681000 Metric tonnes (t) CO2,1.805 m Metric tonnes (t) CO2 1.804 m Metric t...,1.148 m Metric tonnes (t) CO2 1.147 m Metric t...,Not Disclosed,1.123 m Metric tonnes (t) CO2,466000 Metric tonnes (t) CO2,Not Disclosed,123.3 m Metric tonnes (t) CO2 99.2 m Metric to...,20.4 m Metric tonnes (t) CO2 17 m Metric tonne...,Not Disclosed,...,Not Disclosed,Not Disclosed,Not Disclosed,Not Disclosed,Not Disclosed,1 m Metric tonnes (t) CO2 800000 Metric tonnes...,22000 Metric tonnes (t) CO2 19000 Metric tonne...,122000 Metric tonnes (t) CO2 107000 Metric ton...,Not Disclosed,Not Disclosed
2022-12-31,569000 Metric tonnes (t) CO2e,1.69 m Metric tonnes (t) CO2e,663000 Metric tonnes (t) CO2e,Not Disclosed,1.121 m Metric tonnes (t) CO2e,94000 Metric tonnes (t) CO2e,Not Disclosed,97.8 m Metric tonnes (t) CO2e,17.7 m Metric tonnes (t) CO2e,Not Disclosed,...,Not Disclosed,Not Disclosed,Not Disclosed,Not Disclosed,Not Disclosed,800000 Metric tonnes (t) CO2e,57000 Metric tonnes (t) CO2e,107000 Metric tonnes (t) CO2e,Not Disclosed,Not Disclosed
